In [52]:
import pandas as pd
import numpy as np
import json
import random
import itertools
import ast
import json

from collections import defaultdict
%matplotlib inline

In [3]:
dump_fpath = 'data/declarations.json'

with open(dump_fpath) as f:
    declarations = json.loads(f.read())

In [4]:
random.choice(declarations)

{'bonds': [],
 'incomes': [{'comment': '', 'relative': None, 'size': 1192008.62},
  {'comment': '',
   'relative': {'id': 2, 'name': 'Супруг(а)'},
   'size': 153664.0}],
 'main': {'document_type': {'id': 1, 'name': 'Антикоррупционная декларация'},
  'office': {'id': 3962,
   'name': 'УФСИН Тамбовская область',
   'post': 'начальник ФКУ ОК УФСИН России по Тамбовской области',
   'region': {'id': 82, 'name': 'Тамбовская область'},
   'type': {'id': 24, 'name': 'Федеральный, со структурой'},
   'url': 'http://www.68.fsin.su/'},
  'party': None,
  'person': {'family_name': 'Рыбак',
   'given_name': 'Сергей',
   'id': 35106,
   'name': 'Рыбак Сергей Иванович',
   'patronymic_name': 'Иванович'},
  'year': 2015},
 'real_estates': [{'comment': '',
   'country': 'Россия',
   'name': 'земельный участок',
   'own_type': {'id': 20, 'name': 'Индивидуальная'},
   'region': None,
   'relative': None,
   'share': None,
   'square': 26.4,
   'type': {'id': 1, 'name': 'Земельный участок'}},
  {'comment'

In [5]:
columns = ['person_id', 'person_name', 'year', 'office_id',
           'office_name', 'income', 'savings',
           'real_estate_amount',
           'real_estate_squares']

rows = []
for dec in declarations:
    row = [
        dec['main']['person']['id'],
        dec['main']['person']['name'],
        dec['main']['year'],
        dec['main']['office']['id'],
        dec['main']['office']['name'],
    ]
    income_sum = sum([inc['size'] for inc in dec['incomes']])
    savings_sum = sum([float(sav.split('руб.')[0].replace(',', '.').replace(' ', '')) for sav in dec['savings']])
    real_estate_amount = len(dec['real_estates'])
    real_estate_squares_sum = sum([(estate['square']  or 0) for estate in dec['real_estates']])
    row += [income_sum, savings_sum, real_estate_amount, real_estate_squares_sum]
    rows.append(row)

In [6]:
df = pd.DataFrame(rows, columns=columns)
df = df.sort_values(by=['person_id', 'year'])
df.head()


,person_id,person_name,year,office_id,office_name,income,savings,real_estate_amount,real_estate_squares
30357,8,Зюганов Геннадий Андреевич,1998,14,Государственная Дума,124154.00,0.00,1,150.0
31969,8,Зюганов Геннадий Андреевич,1999,449,Президент Российской Федерации,506247.00,0.00,3,252.7
127,8,Зюганов Геннадий Андреевич,2006,14,Государственная Дума,1257784.00,141740.91,1,167.4
1109,8,Зюганов Геннадий Андреевич,2006,449,Президент Российской Федерации,3590668.48,400490.32,2,281.3
5520,8,Зюганов Геннадий Андреевич,2009,14,Государственная Дума,2177821.52,0.00,3,448.7


In [7]:
df.shape

(200545, 9)

In [8]:
df.office_id.value_counts()

450     56469
14       5596
607      3198
1397     1828
5        1486
979      1365
598      1291
3898     1119
482      1104
3958     1078
5953     1058
4         967
873       955
3929      914
3949      860
20        790
5963      780
3906      743
456       717
959       686
3939      653
3932      641
191       633
3944      633
4199      626
3942      601
3974      600
3963      593
3930      583
3950      566
        ...  
4580        1
4802        1
5148        1
4873        1
4669        1
5861        1
5995        1
4541        1
5154        1
4543        1
6299        1
5483        1
4028        1
4642        1
6333        1
6890        1
6563        1
4514        1
4540        1
6378        1
4668        1
4634        1
1854        1
4627        1
4031        1
4842        1
4586        1
6204        1
6361        1
6147        1
Name: office_id, Length: 3239, dtype: int64

In [9]:
df[df.office_id == 607]

,person_id,person_name,year,office_id,office_name,income,savings,real_estate_amount,real_estate_squares
96442,853,Смильгин Виктор Эдуардоваич,2014,607,Министерство сельского хозяйства,1070442.06,0.0,4,1019.20
21690,1429,Гаевский Валерий Вениаминович,2014,607,Министерство сельского хозяйства,9433496.15,0.0,4,674.20
38112,2682,Астраханцева Елена Юрьевна,2015,607,Министерство сельского хозяйства,5583296.76,0.0,3,254.70
69158,2682,Астраханцева Елена Юрьевна,2016,607,Министерство сельского хозяйства,3748343.65,0.0,2,169.80
121289,2682,Астраханцева Елена Юрьевна,2017,607,Министерство сельского хозяйства,3612804.52,0.0,2,169.80
21682,3163,Семенов Павел Владимирович,2013,607,Министерство сельского хозяйства,4159740.12,0.0,12,2867.90
21688,3163,Семенов Павел Владимирович,2014,607,Министерство сельского хозяйства,16535351.20,0.0,14,2997.40
38111,4546,Громыко Евгений Васильевич,2015,607,Министерство сельского хозяйства,14690567.35,0.0,53,75243.01
69159,4546,Громыко Евгений Васильевич,2016,607,Министерство сельского хозяйства,17923271.14,0.0,43,72697.57
121290,4546,Громыко Евгений Васильевич,2017,607,Министерство сельского хозяйства,14076699.84,0.0,45,72952.17


In [30]:
df = pd.read_csv('data/declarations.csv')
df.head()

,person_id,person_name,year,office_id,office_name,income,savings,real_estate_amount,real_estate_squares
0,8,Зюганов Геннадий Андреевич,1998,14,Государственная Дума,124154.00,0.00,1,150.0
1,8,Зюганов Геннадий Андреевич,1999,449,Президент Российской Федерации,506247.00,0.00,3,252.7
2,8,Зюганов Геннадий Андреевич,2006,14,Государственная Дума,1257784.00,141740.91,1,167.4
3,8,Зюганов Геннадий Андреевич,2006,449,Президент Российской Федерации,3590668.48,400490.32,2,281.3
4,8,Зюганов Геннадий Андреевич,2009,14,Государственная Дума,2177821.52,0.00,3,448.7


In [13]:
df.to_csv('data/declarations.csv', index=False)

In [10]:
grouped_view = df.groupby(['person_id', 'person_name', 'year']).first()

In [53]:
common_offices = df.groupby(['year', 'office_id'])['person_id'].apply(list).reset_index()
common_offices.head()

,year,office_id,person_id
0,1998,14,"[8, 68, 89, 132, 140, 144, 147, 150, 152, 177,..."
1,1999,449,"[8, 89, 177, 580, 582, 960, 9214, 19333, 19337..."
2,2006,14,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
3,2006,449,"[8, 89, 542, 584]"
4,2008,12,[1383]


In [54]:
common_offices.to_csv('data/common_offices.csv', index=False)

In [12]:
worked_at = {
    # person_id: { year: office_id }
}
for dec in declarations:
    if not worked_at.get(dec['main']['person']['id']):
        worked_at[dec['main']['person']['id']] = {}
    worked_at[dec['main']['person']['id']][dec['main']['year']] = dec['main']['office']['id']

In [13]:
len(worked_at)

132016

In [14]:
def to_arr(adict):
    columns = [1998,1999,2006,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]
    arr = np.array([adict.get(c, np.NaN) for c in columns])
    return arr

worked_at = {
    k: to_arr(v) for k, v in worked_at.items()   
}

In [19]:
TARGET_OFFICE = 14
TARGET_YEAR = 2018
target_persons = df[(df.year == TARGET_YEAR) & (df.office_id == TARGET_OFFICE)].person_id.unique()
pairs = itertools.combinations(target_persons, 2)
with open(f'worked_together_{TARGET_OFFICE}_{TARGET_YEAR}.csv', 'w') as f:
    for pair in pairs:
        guy1, guy2 = pair
        count_worked_together = (worked_at[guy1] == worked_at[guy2]).sum()
        #for year in worked_at[guy1].keys():
        #    if (worked_at[guy1].get(year) and worked_at[guy2].get(year) and
        #        worked_at[guy1].get(year) == worked_at[guy2].get(year)):
        #        count_worked_together += 1
        if count_worked_together:
            f.write(f'{guy1};{guy2};{count_worked_together}\n')

In [ ]:
worked_together = {}
i = 0
with open(f'worked_together.csv', 'w') as f:
    for year in common_offices.year.unique():
        for _, office, persons in common_offices[common_offices.year == year].values:
            for pair in itertools.combinations(persons, 2):
                worked_together.setdefault(pair, 0)
                worked_together[pair] +=1
        
        for k,v in worked_together.items():
            f.write(f'{year};{k[0]};{k[1]};{v}\n')
        worked_together = {}
        i += 1
        print(i / len(common_offices))
        print(year, 'done')

0.00011674060238150829
1998 done
0.00023348120476301658
1999 done
0.0003502218071445249
2006 done
0.00046696240952603317
2008 done
0.0005837030119075414
2009 done
0.0007004436142890498
2010 done
0.000817184216670558
2011 done
0.0009339248190520663
2012 done
0.0010506654214335746
2013 done


In [ ]:
for year in common_offices.year.unique():
    with open(f'worked_together_{year}.csv', 'w') as f:
        target_persons = df[(df.year == year)].person_id.unique()
        pairs = itertools.combinations(target_persons, 2)
        for pair in pairs:
            guy1, guy2 = pair
            count_worked_together = (worked_at[guy1] == worked_at[guy2]).sum()
            if count_worked_together:
                f.write(f'{year};{guy1};{guy2};{count_worked_together}\n')
    print(year, 'done')

1998 done
1999 done
2006 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done


## Граф связей

In [20]:
YEARS = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
OFFICES_EXCLUDE = [450]

In [21]:
common_offices = pd.read_csv('data/common_offices.csv')
common_offices.head()

,year,office_id,person_id
0,1998,14,"[8, 68, 89, 132, 140, 144, 147, 150, 152, 177,..."
1,1999,449,"[8, 89, 177, 580, 582, 960, 9214, 19333, 19337..."
2,2006,14,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,..."
3,2006,449,"[8, 89, 542, 584]"
4,2008,12,[1383]


In [22]:
year_co = common_offices[common_offices['year'].isin(YEARS)].copy()
year_co = year_co[~year_co['office_id'].isin(OFFICES_EXCLUDE)]
year_co.head()

,year,office_id,person_id
260,2010,1,"[292, 294, 295, 296, 297, 298, 304, 857, 1021,..."
261,2010,3,"[679, 680, 681, 682, 7300, 7302, 7304, 7305, 7..."
262,2010,4,[5350]
263,2010,7,"[1882, 19701]"
264,2010,10,"[19803, 19804, 19805, 19807, 19809, 19810, 198..."


In [23]:
def to_pair_dict(persons):
    persons = sorted(ast.literal_eval(persons))
    for pair in itertools.combinations(persons, 2):
        graph[pair] += 1
    
graph = defaultdict(int)    
_ = year_co['person_id'].map(lambda x: to_pair_dict(x))

In [24]:
len(graph)

7695471

In [28]:
with open('data/graph.csv', 'w') as graph_file:
    for pair in sorted(graph):
        graph_file.write('{},{},{}\n'.format(pair[0], pair[1], graph[pair]))

## Справочник доходов депутатов

In [119]:
def sum_diff_offices(g):
    summed = {}
    if len(g) > 1:
        print(g)

In [141]:
# Если несколько записей на одного депутата па один год на один офис, то берем первую
officials_year = (df
                    .groupby(by=['year', 'person_id', 'person_name', 'office_id'])['income']
                    .first()
                    .to_frame('income')
                    .reset_index())
# Суммируем доход в разных офисах
officials_income = (officials_year
                    .groupby(by=['person_id', 'person_name'])['income']
                    .sum()                    
                    .to_frame('income')
                    .reset_index())
officials_income = officials_income.set_index('person_id')
officials_income.head()

,person_name,income
person_id,,
8,Зюганов Геннадий Андреевич,5.791525e+07
9,Алферов Жорес Иванович,1.601186e+08
10,Андреев Андрей Анатольевич,2.962840e+07
11,Апарина Алевтина Викторовна,1.049138e+07
12,Афонин Юрий Вячеславович,6.922374e+07


In [148]:
officials_income.to_json('data/officials.json', orient='index', force_ascii=False, double_precision=2)